In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.metrics import mean_squared_error

#from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv('C:/Users/HP/Desktop/pro/main_dataframe.csv')
df.head(2)

,adult,id,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,genre_name,collection_name,production_companies_name,keywords,cast_name,crew_name,vote_average
0,False,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,"['Animation', 'Comedy', 'Family']",Toy Story Collection,['Pixar Animation Studios'],"['jealousy', 'toy', 'boy', 'friendship', 'frie...","['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim...","['John Lasseter', 'Joss Whedon', 'Andrew Stant...",7.7
1,False,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,"['Adventure', 'Fantasy', 'Family']",NaN,"['TriStar Pictures', 'Teitler Film', 'Intersco...","['board game', 'disappearance', ""based on chil...","['Robin Williams', 'Jonathan Hyde', 'Kirsten D...","['Larry J. Franco', 'Jonathan Hensleigh', 'Jam...",6.9


In [3]:
df.shape

(45463, 17)

In [4]:
links_small = pd.read_csv('links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [5]:
df = df.drop([19730, 29503, 35587])

In [6]:
df['id'] = df['id'].astype('int')

In [7]:
smd = df[df['id'].isin(links_small)]
smd.shape

(9098, 17)

In [8]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [9]:
id_map = pd.read_csv('links_small.csv')[['movieId', 'tmdbId']]
id_map.head(2)

,movieId,tmdbId
0,1,862.0
1,2,8844.0


In [10]:
#id_map = pd.read_csv('C:/Users/WA464JA/OneDrive - EY/Desktop/RECSYS/Capstone-Project_Movie-Recommendation-Engine-main/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(df[['original_title', 'id']], on='id')#.set_index('title')

In [11]:
smd.isnull().sum()

adult                           0
id                              0
original_language               0
original_title                  0
overview                       12
popularity                      0
release_date                    0
revenue                         0
runtime                         0
status                          2
genre_name                      0
collection_name              7424
production_companies_name       0
keywords                      764
cast_name                       0
crew_name                       0
vote_average                    0
dtype: int64

In [12]:
keywords = pd.read_csv('keywords.csv')

In [13]:
keywords.dropna(inplace=True)

In [14]:
keywords['id'] = keywords['id'].astype('int')

In [15]:
#df = df.merge(credits, on='id')
df = df.merge(keywords, on='id')

In [16]:
smd = df[df['id'].isin(links_small)]
smd.shape

(9150, 18)

In [17]:
smd.isnull().sum()

adult                           0
id                              0
original_language               0
original_title                  0
overview                       12
popularity                      0
release_date                    0
revenue                         0
runtime                         0
status                          2
genre_name                      0
collection_name              7470
production_companies_name       0
keywords_x                    771
cast_name                       0
crew_name                       0
vote_average                    0
keywords_y                      0
dtype: int64

In [18]:
smd['keywords_y'] = smd['keywords_y'].apply(literal_eval)

In [19]:
smd['keywords_y'] = smd['keywords_y'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [20]:
s = smd.apply(lambda x: pd.Series(x['keywords_y']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [21]:
s = s.value_counts()
s[:5]

independent film        606
woman director          546
murder                  399
duringcreditsstinger    327
based on novel          313
Name: keyword, dtype: int64

In [22]:
s = s[s > 1]

In [23]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [24]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [25]:
smd['keywords_y'] = smd['keywords_y'].apply(filter_keywords)
smd['keywords_y'] = smd['keywords_y'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords)y'] = smd['keywords_y'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [26]:
smd.head()

,adult,id,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,genre_name,collection_name,production_companies_name,keywords_x,cast_name,crew_name,vote_average,keywords_y,keywords)y
0,False,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,"['Animation', 'Comedy', 'Family']",Toy Story Collection,['Pixar Animation Studios'],"['jealousy', 'toy', 'boy', 'friendship', 'frie...","['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim...","['John Lasseter', 'Joss Whedon', 'Andrew Stant...",7.7,"[jealousi, toy, boy, friendship, friend, rival...","[jealousi, toy, boy, friendship, friend, rival..."
1,False,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,"['Adventure', 'Fantasy', 'Family']",NaN,"['TriStar Pictures', 'Teitler Film', 'Intersco...","['board game', 'disappearance', ""based on chil...","['Robin Williams', 'Jonathan Hyde', 'Kirsten D...","['Larry J. Franco', 'Jonathan Hensleigh', 'Jam...",6.9,"[board gam, disappear, based on children's boo...","[boardgam, disappear, basedonchildren'sbook, n..."
2,False,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,"['Romance', 'Comedy']",Grumpy Old Men Collection,"['Warner Bros.', 'Lancaster Gate']","['fishing', 'best friend', 'duringcreditssting...","['Walter Matthau', 'Jack Lemmon', 'Ann-Margret...","['Howard Deutch', 'Mark Steven Johnson', 'Mark...",6.5,"[fish, best friend, duringcreditssting]","[fish, bestfriend, duringcreditssting]"
3,False,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,"['Comedy', 'Drama', 'Romance']",NaN,['Twentieth Century Fox Film Corporation'],"['based on novel', 'interracial relationship',...","['Whitney Houston', 'Angela Bassett', 'Loretta...","['Forest Whitaker', 'Ronald Bass', 'Ronald Bas...",6.1,"[based on novel, interracial relationship, sin...","[basedonnovel, interracialrelationship, single..."
4,False,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,['Comedy'],Father of the Bride Collection,"['Sandollar Productions', 'Touchstone Pictures']","['baby', 'midlife crisis', 'confidence', 'agin...","['Steve Martin', 'Diane Keaton', 'Martin Short...","['Alan Silvestri', 'Elliot Davis', 'Nancy Meye...",5.7,"[babi, midlife crisi, confid, age, daughter, m...","[babi, midlifecrisi, confid, age, daughter, mo..."


In [27]:
smd['genre_name'] = smd['genre_name'].apply(literal_eval)

In [28]:
smd['cast_name'] = smd['cast_name'].apply(literal_eval)

In [29]:
smd['crew_name'] = smd['crew_name'].apply(literal_eval)

In [30]:
smd['genre_name'][0]

['Animation', 'Comedy', 'Family']

In [ ]:
#Content based filtering

In [31]:
smd['soup']=smd['genre_name']+smd['cast_name']+smd['crew_name']+list(smd['keywords_y'])

In [32]:
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [33]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [34]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [35]:
smd = smd.reset_index()
titles = smd['original_title']
indices = pd.Series(smd.index, index=smd['original_title'])

In [36]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [37]:
title='The Godfather'

In [38]:
get_recommendations('The Godfather').head(10)

982              The Godfather: Part II
1590            The Godfather: Part III
969                      Apocalypse Now
2986                   The Conversation
3685                    The Cotton Club
775     Around the World in Eighty Days
4486                 One from the Heart
1679                      The Outsiders
5827                        Rumble Fish
2364                      South Pacific
Name: original_title, dtype: object

In [39]:
get_recommendations('The Dark Knight').head(10)

7971                              The Dark Knight Rises
6174                                      Batman Begins
6575                                       The Prestige
8750                                     Jurassic World
3323                                         15 Minutes
6188                                  War of the Worlds
4505                                          Daredevil
5564                                         Collateral
3466                       A.I. Artificial Intelligence
4903    Master and Commander: The Far Side of the World
Name: original_title, dtype: object

In [40]:
smd.drop('collection_name',axis=1,inplace=True)

In [41]:
########## Collaborative filtering
import surprise

In [42]:
reader = surprise.Reader()

In [43]:

ratings = pd.read_csv('ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [44]:
from surprise import SVD,Reader,Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
ratings =  pd.read_csv("ratings_small.csv")
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
kf = KFold(n_splits=5)
kf.split(data)

<generator object KFold.split at 0x00000209531CF740>

In [45]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8940  0.8965  0.8988  0.9030  0.8882  0.8961  0.0049  
MAE (testset)     0.6892  0.6910  0.6919  0.6935  0.6845  0.6900  0.0031  
Fit time          0.97    0.97    0.97    0.98    0.99    0.98    0.01    
Test time         0.11    0.12    0.22    0.12    0.11    0.14    0.04    


{'test_rmse': array([0.89402443, 0.89651112, 0.89880489, 0.90295496, 0.8881786 ]),
 'test_mae': array([0.68919363, 0.69100958, 0.69188443, 0.69349738, 0.68451132]),
 'fit_time': (0.965803861618042,
  0.9685196876525879,
  0.9685194492340088,
  0.9841151237487793,
  0.9882588386535645),
 'test_time': (0.1093449592590332,
  0.12494158744812012,
  0.21869611740112305,
  0.12496781349182129,
  0.10934567451477051)}

In [46]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [47]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.7385616239294306, details={'was_impossible': False})

In [48]:
indices_map = id_map.set_index('id')

In [49]:
smd.head()

,index,adult,id,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,genre_name,production_companies_name,keywords_x,cast_name,crew_name,vote_average,keywords_y,keywords)y,soup
0,0,False,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,"[Animation, Comedy, Family]",['Pixar Animation Studios'],"['jealousy', 'toy', 'boy', 'friendship', 'frie...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[John Lasseter, Joss Whedon, Andrew Stanton, J...",7.7,"[jealousi, toy, boy, friendship, friend, rival...","[jealousi, toy, boy, friendship, friend, rival...",Animation Comedy Family Tom Hanks Tim Allen Do...
1,1,False,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,"[Adventure, Fantasy, Family]","['TriStar Pictures', 'Teitler Film', 'Intersco...","['board game', 'disappearance', ""based on chil...","[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Larry J. Franco, Jonathan Hensleigh, James Ho...",6.9,"[board gam, disappear, based on children's boo...","[boardgam, disappear, basedonchildren'sbook, n...",Adventure Fantasy Family Robin Williams Jonath...
2,2,False,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,"[Romance, Comedy]","['Warner Bros.', 'Lancaster Gate']","['fishing', 'best friend', 'duringcreditssting...","[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Howard Deutch, Mark Steven Johnson, Mark Stev...",6.5,"[fish, best friend, duringcreditssting]","[fish, bestfriend, duringcreditssting]",Romance Comedy Walter Matthau Jack Lemmon Ann-...
3,3,False,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,"[Comedy, Drama, Romance]",['Twentieth Century Fox Film Corporation'],"['based on novel', 'interracial relationship',...","[Whitney Houston, Angela Bassett, Loretta Devi...","[Forest Whitaker, Ronald Bass, Ronald Bass, Ez...",6.1,"[based on novel, interracial relationship, sin...","[basedonnovel, interracialrelationship, single...",Comedy Drama Romance Whitney Houston Angela Ba...
4,4,False,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,[Comedy],"['Sandollar Productions', 'Touchstone Pictures']","['baby', 'midlife crisis', 'confidence', 'agin...","[Steve Martin, Diane Keaton, Martin Short, Kim...","[Alan Silvestri, Elliot Davis, Nancy Meyers, N...",5.7,"[babi, midlife crisi, confid, age, daughter, m...","[babi, midlifecrisi, confid, age, daughter, mo...",Comedy Steve Martin Diane Keaton Martin Short ...


In [104]:
#hybrid filtering

In [50]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map[id_map['original_title'] == title]['id'].values[0]
    #print(idx)
    movie_id = id_map[id_map['original_title'] == title]['movieId'].values[0]
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['original_title', 'vote_average','genre_name' ,'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [51]:
hybrid(500, 'Father of the Bride Part II')

,original_title,vote_average,genre_name,id,est
8978,The Intern,7.1,[Comedy],257211,3.455421
6595,The Holiday,6.7,"[Comedy, Romance]",1581,3.351483
1675,Dead Men Don't Wear Plaid,6.5,"[Comedy, Mystery]",9442,3.281626
3407,The World According to Garp,6.9,[Comedy],11307,3.239328
4308,The Four Seasons,6.3,"[Comedy, Drama, Romance]",25113,3.220658
5038,Something's Gotta Give,6.3,"[Drama, Comedy, Romance]",6964,3.217175
2180,Drop Dead Gorgeous,6.5,[Comedy],10490,3.216455
3223,What Women Want,6.1,"[Comedy, Romance]",3981,3.198183
8466,Philomena,7.4,[Drama],205220,3.151926
4833,Bugsy Malone,6.5,"[Drama, Action, Comedy, Music, Family]",8446,3.151128


In [52]:
id_map[id_map['original_title'] == 'Philomena']

,movieId,id,original_title
8430,106438,205220.0,Philomena


In [83]:
r[r['movieId']==106438]['rating'].sum()/r[r['movieId']==106438]['rating'].count()

3.747912317327766

In [54]:
h=hybrid(500, 'Father of the Bride Part II')['est']
rmse=[]

In [55]:
r=pd.read_csv('ratings.csv')

In [57]:
smd.to_csv('smd.csv')

In [58]:
for j in range(5):
    user=int(input('Enter the user'))
    inp=input('Enter the movie')
    h=hybrid(user, inp)['est']
    li=list(hybrid(user, inp)['original_title'].values)
    li1=[]
    for i in li:
    #print(i)
        mov = id_map[id_map['original_title'] == i]['movieId'].values[0]
        #print(mov)
        li1.append(r[r['movieId']==mov]['rating'].sum()/r[r['movieId']==mov]['rating'].count())
    rmse.append(np.sqrt(mean_squared_error(li1,h)))
    #print(li1)
    #print(h)
np.sum(rmse)/5
    

Enter the user6789
Enter the movieIt Takes Two
Enter the user45
Enter the movieLe confessionnal
Enter the user4987
Enter the movieLa Haine
Enter the user1
Enter the movieCasper
Enter the user678
Enter the movieThe War


0.6085634296770952

In [60]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
